# Project: Ford GoBike System Data
## (January 2018 - December 2018)

### by Mohamed.elhaddad 

## Table of Contents
- [Introduction](#intro)
- [wrangling](#wrangling)
- [Cleaning](#clean)
- [Univariate Exploration](#univariate)
- [Bivariate Explorationn](#bivariate)
- [Multivariate Exploration](#multivariate)
- [Summary](#summary)

<a id='intro'></a>
## Introduction
> This document explores a data set containing UFord GoBike System Data. There are approximately "174952" rides made
in a bike-sharing system covering the greater San Francisco
Bay area in the set.


<a id='wrangling'></a>
## Data Wrangling
> In this step we will import libraries wich will help us in our analysis and explore our data to know what operation we need to do to make our data clean and tidy .

In [ ]:
# importing important libraries 
import pandas as pd 
import numpy as np 
import seaborn as sb 
import matplotlib.pyplot as plt 
from datetime import datetime, timedelta
%matplotlib inline 

In [ ]:
# read data_set and make copy to save it 
df_r = pd.read_csv("../input/ford-gobike-2019feb-tripdata/201902-fordgobike-tripdata.csv")
df = df_r.copy()

In [ ]:
df.sample(10)

In [ ]:
# showing important information 
df.info()

In [ ]:
#getting some additional description for stats figures
df.describe()

In [ ]:
# find duplicates
df.duplicated().sum()
# No duplicates here 

In [ ]:
# What about NaN values?
df.isnull().sum() 

<a id='clean'></a>
## Inrtodutory cleaning 
there are some quality issues that need to be cleaned :
- missing values(start_station_id, start_station_name, end_station_id, end_station_name, member_birth_year, member_gender)
- Start/end times are not in the timestamp format
- The following columns are not in object format: 
  - start_station_id
  - end_station_id
  - bike_id
- Member birth is in the float format, but should be converted to integer format

In [ ]:
# Drop missing values 
df.dropna(inplace=True)

In [ ]:
# Changing the format to datetime
df['start_time'] = pd.to_datetime(df['start_time'])
df['end_time'] = pd.to_datetime(df['end_time'])

In [ ]:
# Changing the user_type, member_gender, and bike_share_for_all_trip to category format
df.user_type = df.user_type.astype('category')
df.member_gender = df.member_gender.astype('category')
df.bike_share_for_all_trip = df.bike_share_for_all_trip.astype('category')

In [ ]:
# Changing the bike_id, start_station_id, and end_station_id to object
df['start_station_id'] = df['start_station_id'].astype(str)
df['end_station_id'] = df['end_station_id'].astype(str)
df['bike_id'] = df['bike_id'].astype(str)

In [ ]:
# getting the ages of the members and convert it to integer 
df['Age'] = 2021 - df['member_birth_year']
df['Age'] = df['Age'].astype(int)

In [ ]:
# converting it to integer
df.member_birth_year = df.member_birth_year.astype(int)

In [ ]:
# what is the average of ages ? 
df.Age.mean()

In [ ]:
df.Age.describe(percentiles = [.99])
# So it looks like 99% of users are under the age of 66

In [ ]:
# Only keep the events where riders were under 66 years of age
df = df.query('Age <= 66')

In [ ]:
# Start time month (January - December)
df['start_time_month'] = df['start_time'].dt.strftime('%B')

In [ ]:
# Start time month (1-12)
df['start_time_month_num'] = df['start_time'].dt.month.astype(int)

In [ ]:
# Start time weekday
df['start_time_weekday'] = df['start_time'].dt.strftime('%A')

In [ ]:
# Start and end time hour
df['start_time_hour'] = df['start_time'].dt.hour
df['end_time_hour'] = df['end_time'].dt.hour

In [ ]:
# Duration in seconds to duration in minutes
df['duration_min'] = df['duration_sec']/60
df['duration_min'] = df['duration_min'].astype(int)

In [ ]:
# Not important columns 
df.drop(['start_station_latitude', 'start_station_longitude','end_station_latitude','end_station_longitude', 'member_birth_year'], axis=1, inplace=True)

### What is the structure of your dataset?

> There are 174952 columns for Ford GoBike System in a bike-sharing system wich covering the greater San Francisco
Bay area,  with 17 features, including duration_sec, start_time, end_time, end_time, start_station_id, start_station_nam,  start_station_latitude  start_station_longitude     end_station_id            end_station_name          end_station_latitude     end_station_longitude, bike_id, user_type, member_birth_year, member_gender, bike_share_for_all_trip, Age and month .           

### What is/are the main feature(s) of interest in your dataset?

> The main features include data related to the start and end of a ride. This can be used to calculate when bikes are in high or low demand. There is also ample data related to the users of the bike service including their age, gender, and whether they are a paying member or a casual user.


### What features in the dataset do you think will help support your investigation into your feature(s) of interest?

> The start and end times, and related time-based information will be be usedheavily

<a id='univariate'></a>
## Univariate Exploration

> In this section,  I investigate the distributions of individual variables of interest. If
you see unusual points or outliers, take a deeper look to clean things up
and prepare yourself to look at relationships between variables.

In [ ]:
# Setting color 
color = sb.color_palette()[0]

### What is the ditripution of week-day usage? 

In [ ]:
# Weekday usage of the Ford Go bike system
weekday = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

plot = sb.catplot(data=df, x='start_time_weekday', kind='count', color = color, order = weekday)

plot.set_axis_labels('Weekdays', 'Number of Bike Trips')

plot.fig.suptitle('Ford GoBike Usage by Weekday', y=1.05, fontsize=16, fontweight='bold');

plot.set_xticklabels(rotation=75);

>Observation(1): Tuesdays and Wednesdays are the most popular days for using the bike sharing system, however Thursdays, Mondays and Fridays are very close to the Tues/Weds numbers , saturday and sunday are not popular as the rest of days.

### What is the ditripution useage by hour ? 

In [ ]:
# Hourly usage of the Ford Go Bike System
plot = sb.catplot(data = df, x='start_time_hour', kind='count', color = color, aspect = 1.7)
plot.set_axis_labels('Hours', 'Number of Bike Rides')
plot.fig.suptitle('Ford GoBike System Usage by Hour', y=1.05, fontsize=16, fontweight='bold');

> Observation(2): The most usage during the morning hours of 8-9am, and in the afternoon hours of 5-6pm

### What is the ditripution of Ages  ? 

In [ ]:
# Age group data
bin_edges = np.arange(0, 65, 5)
plt.hist(data = df, x = 'Age', bins = bin_edges);
plt.title("Ford GoBike System User by Age", y=1.05, fontsize=16, fontweight='bold')
plt.xlabel('Age of Members (Years)')
plt.ylabel('Number of Bike Trips');

>Observation(3): The mean age of a user is 35.5 years old.

### What is the ditripution of Gender ? 

In [ ]:
# Gender data 
o = df.member_gender.value_counts().index
sb.countplot(data=df , x='member_gender' , color=color , order=o);

>Observation(4): Most of the users of this system are male.

### What is the ditripution of user type  ? 

In [ ]:
# Types of users 
o = df.user_type.value_counts().index
sb.countplot(data=df , x='user_type' , color=color , order=o);

> Observation(5): Most of the users of this system are subscribers.

### What is the most popular Start stations ? 

In [ ]:
# Most popular Start station 
fig_dims = (12, 12)
fig, ax = plt.subplots(figsize=fig_dims)
c = sb.color_palette()[0]
o = df.start_station_name.value_counts().head(50).index
sb.countplot(data=df , y='start_station_name' , color=c , order=o);

# Putting percentage  
start_counts = df.start_station_name.value_counts().head(50)
for i in range(start_counts.shape[0]):
  count = start_counts[i]
  pct_string = '{:0.1f}%'.format(100*count/df.shape[0])
  plt.text(count+1, i, pct_string, va = 'center');

> Observation(6): The most popular stations to start are Market St and San francisco caltrian station 2 

## Discuss the distribution(s) of your variable(s) of interest. Were there any unusual points? Did you need to perform any transformations?

> Thanks to the - [Cleaning](#clean)  process, discovering the parameter distributions was extremely easy


### Of the features you investigated, were there any unusual distributions? Did you perform any operations on the data to tidy, adjust, or change the form of the data? If so, why did you do this?

> The distribution of the trip duration was a little surprising at first plotting out the original dataو Nothing seems out of the normal.


<a id='bivariate'></a>
## Bivariate Exploration

> In this section, I will investigate relationships between pairs of variables in the flights
data. First, I am going to make a data frame that include only the data need for this study. I will also create calculated columns in this data frame so that I can use it in multivariate exploration as well.

### What is the relation between week-day and number  of trips depending on  user type  ? 

In [ ]:
# Customer Usage by Weekday vs. Subscriber Usage by Weekday
plt.figure(figsize=(12, 5))

df_user_week = df.groupby(['start_time_weekday', 'user_type']).size().reset_index()
weekday = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

ax = sb.pointplot(data=df_user_week, x='start_time_weekday', y=0, hue = 'user_type', scale=.7, order = weekday);

plt.title('Ford GoBike System Trends by User Type', y=1.05, fontsize=16, fontweight='bold')
plt.xlabel('Weekdays')
plt.ylabel('Number of Bike Trips');
plt.grid()

>Observation(1): Customers have a relatively low usage of the bike share system with a small increase on the weekends. Subscribers are the opposite

### What is the relation between Ages and Time of trip  ? 

In [ ]:
# plotting relation between ages and time tripping 
plt.scatter(data=df, x='duration_min' ,y='Age', alpha=5/20);
plt.xlabel('Duration Time (minutes)')
plt.ylabel('Ages');

> Observation(2): we notice that all of ages prefer to spend thier trips less than 200 minutes 

### What is the relation between ages and type of user   ? 

In [ ]:
# Customer vs Subscriber Usage by Age
plot = sb.catplot(data = df, x = 'Age', col='user_type', kind='count', sharey = False, color = color, height=5, aspect=2)
plot.set_axis_labels('Member Age', 'Number of Bike Trips')
plot.set_titles('{col_name}')
plot.fig.suptitle('Ford GBike System - Customers vs Subscribers Ages', y=1.05, fontsize=16, fontweight='bold');
plot.set_xticklabels(rotation=75);

>Observation(3): Subscribers have a wider range of ages while Customers tends to have mostly younger users (most are und the 31 threshold) 

### What is the relation between Age and Gender  ? 


In [ ]:
# plotting violin 
sb.violinplot(data=df, x='member_gender', y='Age', color=color, innner=None);

In [ ]:
# plotting box plot 
sb.boxplot(data=df, x='member_gender', y='Age', color=color);

>Observation(4): Though not a huge difference, male riders tend to have shorter trips compared to female users, indicated by both a smaller median and shorter IQR.  


### What is the distriputaion of start hour depending on user type   ? 

In [ ]:
# clustered bar chart, absolute counts
plt.figure(figsize = [20, 5])
plt.subplot(1, 2, 1)
sb.countplot(data = df, x = 'start_time_hour', hue = 'user_type')
plt.legend();
plt.title('User Type wise Start Hour');


> Observation(5): Subscribers take trips mostly during 8-9 am and 4-6 pm which is usually working hours of a company.

### What is the relation user type and time of trip  ? 

In [ ]:
# making the order of user type 
o = df.user_type.value_counts().index
sb.barplot(y='duration_sec' , x='user_type' ,data=df , color=color , order=o);

> Observation(6): Customers users spend more time than subscriber users 

### Talk about some of the relationships you observed in this part of the investigation. How did the feature(s) of interest vary with other features in the dataset?

> There are a lot more subscriber usage than customers. The riding habit/pattern varies a lot between subscribers and customers. Subscribers use the bike sharing system for work commnute thus most trips were on work days (Mon-Fri) and especially during rush hours (when going to work in the morning and getting off work in the afternoon), whereas customers tend to ride for fun in the afternoon or early evenings over weekends.

### Did you observe any interesting relationships between the other features (not the main feature(s) of interest)?

> It is interesting to see that subscribers are slightly older than customers on average but take much shorter/quicker rides. And October had the most bike riding usage for subscribers throughout the year.

<a id='multivariate'></a>
## Multivariate Exploration

> I created plots of three or more variables to investigate the Ford Go Bike data even further.

### What is the average trip duration variation in weekdays between customers and subscribers ?

In [ ]:
sb.pointplot(data=df, x='start_time_weekday', y='duration_min', hue='user_type', dodge=0.3, linestyles="");
plt.xlabel('Day of Week');
plt.ylabel('Avg. Trip Duration in Minute');

>Observation(1): It can be seen from the above plot that subscribers ride much shorter/quicker trips compared to customers on each day of the week.

### what is the relation between trips taken by customers and subscribers on weekdays and weekends ? 

In [ ]:
plt.figure(figsize = (16,8));
plt.subplot(2, 1, 1)
customers = df.query('user_type == "Customer"')
ct_counts = customers.groupby(['start_time_weekday', 'start_time_hour']).size()
ct_counts = ct_counts.reset_index(name='count')
ct_counts = ct_counts.pivot(index='start_time_weekday', columns='start_time_hour', values='count')
sb.heatmap(ct_counts, cmap='YlGnBu');
plt.title('Customer', loc='right');
plt.xlabel('Hour of Day');
plt.ylabel('Day of Week');


plt.suptitle('Hourly Usage during Weekdays for Customers and Subscribers', y=1.05, fontsize=16, fontweight='bold');

plt.subplot(2, 1, 2);
subscribers = df.query('user_type == "Subscriber"')
st_counts = subscribers.groupby(['start_time_weekday', 'start_time_hour']).size()
st_counts = st_counts.reset_index(name='count')
st_counts = st_counts.pivot(index='start_time_weekday', columns='start_time_hour', values='count')
sb.heatmap(st_counts, cmap='YlGnBu');
plt.title('Subscriber', loc='right');
plt.xlabel('Hour of Day');
plt.ylabel('Day of Week');

>Observation(2): we can see clearly the trips taken by customers and subscribers on weekdays and weekends.

### Talk about some of the relationships you observed in this part of the investigation. Were there features that strengthened each other in terms of looking at your feature(s) of interest?

The multivariate exploration strengthened some of the patterns discovered in the previous bivariate exploration as well as univariate exploration, the relationship between the multiple variables plotted are visualized altogether and information are presented combined. The efficient/short period of usage for subscribers corresponds to their high concentration on rush hours Monday through Friday, indicating the use is primarily for work commute. The more relaxing and flexible pattern of customer use shows that they're taking advantage of the bike sharing system quite differently from the subscribers, heavily over weekends and in the afternoon, for city tour or leisure purpose probably.        

### Were there any interesting or surprising interactions between features?

The interactions between features are all supplementing each other and quite make sense when looked at combined, there's no big surprise observed. The usage habit difference between male and female is not that much or obvious throughout the exploration, which could be related to the imbalanced number of female riders/records compared to male ones. It'd be interesting to see how male and female use the system differently if there were more female data.    

<a id='summary'></a>
## Summary

- Renting a bike from theFord GoBike System is a fantastic (healthy and environmentally friendly) way of moving around in the city, both for enjoyment and work. There are two types of clients using the system: Subscribers and Customers. Subscribers are primarily daily commuters, having short trips to and from work, who rent a bike on weekdays at 8-9am and 5-6pm, and, occasionally around lunch time. Customers are usually tourists or occassional riders who use the system mainly on weekends to explore the Bay Area.